
1) CHUNKING

2) POS - TAGGING

3) CONSTRUCTION OF CFG TREE

4) DISPLAYING FREQUENCY OF EACH TERM

5) DISPLAYING HIGHEST FREQUENCY TERM

6) DISPLAYING HIGHLY OCCURING BI-GRAM

DATASET USED: **PUBMed 200K RCT**

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct.git
!ls pubmed-rct

Cloning into 'pubmed-rct'...
remote: Enumerating objects: 39, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 39 (delta 8), reused 5 (delta 5), pack-reused 25
Receiving objects: 100% (39/39), 177.08 MiB | 14.73 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (13/13), done.
PubMed_200k_RCT				       PubMed_20k_RCT_numbers_replaced_with_at_sign
PubMed_200k_RCT_numbers_replaced_with_at_sign  README.md
PubMed_20k_RCT


In [ ]:
data_dir = "pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

In [ ]:
def get_lines(filename):
  with open(filename, "r") as f:
    return f.readlines()

In [ ]:
train_lines = get_lines(data_dir+"train.txt")
train_lines[:20]

['###24293578\n',
 'OBJECTIVE\tTo investigate the efficacy of @ weeks of daily low-dose oral prednisolone in improving pain , mobility , and systemic low-grade inflammation in the short term and whether the effect would be sustained at @ weeks in older adults with moderate to severe knee osteoarthritis ( OA ) .\n',
 'METHODS\tA total of @ patients with primary knee OA were randomized @:@ ; @ received @ mg/day of prednisolone and @ received placebo for @ weeks .\n',
 'METHODS\tOutcome measures included pain reduction and improvement in function scores and systemic inflammation markers .\n',
 'METHODS\tPain was assessed using the visual analog pain scale ( @-@ mm ) .\n',
 'METHODS\tSecondary outcome measures included the Western Ontario and McMaster Universities Osteoarthritis Index scores , patient global assessment ( PGA ) of the severity of knee OA , and @-min walk distance ( @MWD ) .\n',
 'METHODS\tSerum levels of interleukin @ ( IL-@ ) , IL-@ , tumor necrosis factor ( TNF ) - , and 

In [ ]:
import re

def remove_urls(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

cleaned_lines_urls = [remove_urls(line) for line in train_lines]

for line in train_lines:
    if re.search(r'http\S+|www\S+|https\S+', line):
        cleaned_line = remove_urls(line)
        break

with open("cleaned_train_urls.txt", "w") as f:
    f.writelines(cleaned_lines_urls)

In [ ]:
def remove_punctuation(text):
    return re.sub(r'[^\w\s]', '', text)

cleaned_lines_punctuation = [remove_punctuation(line) for line in cleaned_lines_urls]

for line in cleaned_lines_urls:
    if re.search(r'[^\w\s]', line):
        cleaned_line = remove_punctuation(line)
        break

with open("cleaned_train_punctuation.txt", "w") as f:
    f.writelines(cleaned_lines_punctuation)

In [ ]:
def convert_to_lowercase(text):
    return text.lower()

with open("cleaned_train_urls.txt", "r") as f:
    cleaned_lines_urls = f.readlines()

cleaned_lines_lowercase = [convert_to_lowercase(line) for line in cleaned_lines_urls]

for line in cleaned_lines_urls:
    if any(char.isupper() for char in line):
        cleaned_line = convert_to_lowercase(line)
        break

with open("cleaned_train_lowercase.txt", "w") as f:
    f.writelines(cleaned_lines_lowercase)

### **PART OF SPEECH TAGGING (POS)**

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
with open("cleaned_train_lowercase.txt", "r") as f:
    cleaned_lines = f.readlines()

preprocessed_text = " ".join(cleaned_lines)

sentences = sent_tokenize(preprocessed_text)

for i, sentence in enumerate(sentences[:5]):
    tokens = word_tokenize(sentence)

    pos_tags = pos_tag(tokens)

    print(f"Sentence {i+1}:")
    print("Tokens before POS tagging:")
    print(tokens)

    print("Tokens after POS tagging:")
    print(pos_tags)
    print("\n")

Sentence 1:
Tokens before POS tagging:
['#', '#', '#', '24293578', 'objective', 'to', 'investigate', 'the', 'efficacy', 'of', '@', 'weeks', 'of', 'daily', 'low-dose', 'oral', 'prednisolone', 'in', 'improving', 'pain', ',', 'mobility', ',', 'and', 'systemic', 'low-grade', 'inflammation', 'in', 'the', 'short', 'term', 'and', 'whether', 'the', 'effect', 'would', 'be', 'sustained', 'at', '@', 'weeks', 'in', 'older', 'adults', 'with', 'moderate', 'to', 'severe', 'knee', 'osteoarthritis', '(', 'oa', ')', '.']
Tokens after POS tagging:
[('#', '#'), ('#', '#'), ('#', '#'), ('24293578', 'CD'), ('objective', 'JJ'), ('to', 'TO'), ('investigate', 'VB'), ('the', 'DT'), ('efficacy', 'NN'), ('of', 'IN'), ('@', 'JJ'), ('weeks', 'NNS'), ('of', 'IN'), ('daily', 'JJ'), ('low-dose', 'JJ'), ('oral', 'JJ'), ('prednisolone', 'NN'), ('in', 'IN'), ('improving', 'VBG'), ('pain', 'NN'), (',', ','), ('mobility', 'NN'), (',', ','), ('and', 'CC'), ('systemic', 'JJ'), ('low-grade', 'JJ'), ('inflammation', 'NN'), ('i

### **CHUNKING**

In [ ]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, RegexpParser
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [ ]:
with open("cleaned_train_lowercase.txt", "r") as f:
    cleaned_lines = f.readlines()

preprocessed_text = " ".join(cleaned_lines)
sentences = sent_tokenize(preprocessed_text)
grammar = "NP: {<DT>?<JJ>*<NN>}"
chunk_parser = RegexpParser(grammar)

In [ ]:
for i, sentence in enumerate(sentences[:5]):
    tokens = word_tokenize(sentence)

    pos_tags = pos_tag(tokens)

    chunked_sentence = chunk_parser.parse(pos_tags)

    print(f"Sentence {i+1}:")
    print("Tokens before POS tagging:")
    print(tokens)

    print("Tokens after POS tagging:")
    print(pos_tags)

    print("Chunks after chunking:")
    print(chunked_sentence)
    print("\n")

Sentence 1:
Tokens before POS tagging:
['#', '#', '#', '24293578', 'objective', 'to', 'investigate', 'the', 'efficacy', 'of', '@', 'weeks', 'of', 'daily', 'low-dose', 'oral', 'prednisolone', 'in', 'improving', 'pain', ',', 'mobility', ',', 'and', 'systemic', 'low-grade', 'inflammation', 'in', 'the', 'short', 'term', 'and', 'whether', 'the', 'effect', 'would', 'be', 'sustained', 'at', '@', 'weeks', 'in', 'older', 'adults', 'with', 'moderate', 'to', 'severe', 'knee', 'osteoarthritis', '(', 'oa', ')', '.']
Tokens after POS tagging:
[('#', '#'), ('#', '#'), ('#', '#'), ('24293578', 'CD'), ('objective', 'JJ'), ('to', 'TO'), ('investigate', 'VB'), ('the', 'DT'), ('efficacy', 'NN'), ('of', 'IN'), ('@', 'JJ'), ('weeks', 'NNS'), ('of', 'IN'), ('daily', 'JJ'), ('low-dose', 'JJ'), ('oral', 'JJ'), ('prednisolone', 'NN'), ('in', 'IN'), ('improving', 'VBG'), ('pain', 'NN'), (',', ','), ('mobility', 'NN'), (',', ','), ('and', 'CC'), ('systemic', 'JJ'), ('low-grade', 'JJ'), ('inflammation', 'NN'), ('i

### **CONSTRUCTION OF CFG TREE**

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk import CFG
from nltk.parse import ChartParser

sentence = "patients with knee osteoarthritis received prednisolone"

cfg_grammar = CFG.fromstring("""
  S -> NP VP
  NP -> N | N PP | N PP N
  VP -> V NP | V NP PP
  PP -> P NP
  N -> "patients" | "knee" | "osteoarthritis" | "prednisolone"
  V -> "received"
  P -> "with"
""")

parser = ChartParser(cfg_grammar)
tokens = word_tokenize(sentence)
print("Parse Trees:")
for tree in parser.parse(tokens):
    print(tree)
    tree.pretty_print()


Parse Trees:
(S
  (NP (N patients) (PP (P with) (NP (N knee))) (N osteoarthritis))
  (VP (V received) (NP (N prednisolone))))
                   S                                           
           ________|____________________________                
          NP                                    |              
    ______|__________________                   |               
   |           PP            |                  VP             
   |       ____|___          |            ______|_______        
   |      |        NP        |           |              NP     
   |      |        |         |           |              |       
   N      P        N         N           V              N      
   |      |        |         |           |              |       
patients with     knee osteoarthritis received     prednisolone



### **DISPLAYING THE FREQUENCY OF EACH TERM**

In [ ]:
from collections import Counter
import re

def tokenize(text):
    return word_tokenize(text)

preprocessed_text = ' '.join(cleaned_lines_lowercase)
tokens = tokenize(preprocessed_text)
token_freq = Counter(tokens)

for term, freq in token_freq.items():
    print(f'{term}: {freq}')


Streaming output truncated to the last 5000 lines.
24411682: 1
additon: 1
25838249: 1
25593483: 1
retroview: 10
haustral: 1
retroflexion: 1
retroflexed: 6
24607075: 1
dmean: 1
25326876: 1
24502652: 1
randomize-controlled: 1
shizuoka: 2
multi-antibiotics: 1
/intervention: 1
/control: 1
right-tailed: 1
25295770: 1
25062857: 1
re-test: 1
demographic/clinical: 1
uhdrs: 3
symbol-digit: 1
24726247: 1
high-mortality: 1
no-opv: 4
thymus/weight: 2
sex-differences: 1
24527996: 1
contact-sport: 3
non-contact-sport: 1
non-csps: 4
csps: 4
mtbisport: 3
motor-vehicle-accident-related: 1
mtbimva: 2
undesirability: 1
thann: 1
25726404: 1
chunks: 1
patient-parent: 1
codebook: 1
chunking: 2
25311446: 1
war-related: 1
steps-up: 4
24923533: 1
hnsccs: 1
egesor: 2
26206644: 1
25645425: 1
losers: 1
weight-losing: 1
swls: 2
ow/nw: 1
25417983: 1
procedural-based: 1
vicarious: 3
toolcreated: 1
24985574: 1
shirmer: 1
24248687: 1
neoallto: 1
lapatinib-related: 1
25581656: 1
re-interviewed: 2
phdd: 3
ggtp: 2
246563

### **DISPLAYING HIGHEST FREQUENCY TERM**

In [ ]:
from collections import Counter
import re
from nltk.tokenize import word_tokenize

preprocessed_lines = [
    "patients with knee osteoarthritis received prednisolone",
    "a total of 100 patients with primary knee oa were randomized",
    "outcome measures included pain reduction and improvement in function scores",
    "pain was assessed using the visual analog pain scale",
    "secondary outcome measures included the western ontario and mcmaster universities osteoarthritis index scores"
]

tokens = word_tokenize(' '.join(preprocessed_lines))
token_freq = Counter(tokens)

for term, freq in token_freq.items():
    print(f'{term}: {freq}')

highest_freq_term = token_freq.most_common(1)[0]

print(f'Highest frequency term: {highest_freq_term[0]}')
print(f'Frequency: {highest_freq_term[1]}')


patients: 2
with: 2
knee: 2
osteoarthritis: 2
received: 1
prednisolone: 1
a: 1
total: 1
of: 1
100: 1
primary: 1
oa: 1
were: 1
randomized: 1
outcome: 2
measures: 2
included: 2
pain: 3
reduction: 1
and: 2
improvement: 1
in: 1
function: 1
scores: 2
was: 1
assessed: 1
using: 1
the: 2
visual: 1
analog: 1
scale: 1
secondary: 1
western: 1
ontario: 1
mcmaster: 1
universities: 1
index: 1
Highest frequency term: pain
Frequency: 3


###**DISPLAYING HIGHLY OCCURING BI-GRAM**

In [ ]:
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.util import ngrams

preprocessed_lines = [
    "patients with knee osteoarthritis received prednisolone",
    "a total of 100 patients with primary knee oa were randomized",
    "outcome measures included pain reduction and improvement in function scores",
    "pain was assessed using the visual analog pain scale",
    "secondary outcome measures included the western ontario and mcmaster universities osteoarthritis index scores"
]

tokens = word_tokenize(' '.join(preprocessed_lines))

bigrams = ngrams(tokens, 2)
bigram_freq = Counter(bigrams)
highest_freq_bigram = bigram_freq.most_common(1)[0]

print(f'Highest frequency bigram: {highest_freq_bigram[0]}')
print(f'Frequency: {highest_freq_bigram[1]}')


Highest frequency bigram: ('patients', 'with')
Frequency: 2
